In [ ]:
%matplotlib notebook
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from matplotlib import animation

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

# Diseño de sistemas y filtros IIR

Un filtro FIR de buena calidad puede requerir una gran cantidad de coeficientes

Es posible implementar filtros más eficientes usando recursividad

Esta es la base de los filtros IIR



## Definición de sistema IIR 

Generalizando el sistema FIR para incluir versiones pasadas de la salida y asumiendo $a[0] = 1$ llegamos a 

$$
\begin{align}
y[n] &= b[0] x[n] + b[1] x[n-1] + b[2] x[n-2] + \ldots + b[L] x[n-L]  \nonumber \\
& - a[1] y[n-1] - a[2] y[n-2] - \ldots - a[M] y[n-M] \nonumber \\
&= \sum_{l=0}^{L} b[l] x[n-l] - \sum_{m=1}^{M} a[m] y[n-m]  \nonumber  \\
\sum_{m=0}^{M} a[m] y[n-m] &= \sum_{l=0}^{L} b[l] x[n-l] \nonumber \\
(a * y)[n] &= (b * x)[n], \nonumber
\end{align}
$$

es decir dos convoluciones discretas que definen una **ecuación de diferencias**

Este tipo de sistema se conoce como 
- sistema IIR (infinite impulse response)
- sistema *auto-regresive moving average* (ARMA)
    - autoregresivo de orden M: incluye valores pasados de la salida
    - media movil de orden L+1: pondera el valor presente y pasados de la entrada

El caso particular del sistema FIR se recupera si

$a[m] = 0$ para $m=[1, \ldots, M]$

## Respuesta en frecuencia del sistema IIR

Aplicando la transformada de Fourier convertimos las convoluciones en multiplicaciones y encontramos la respuesta en frecuencia como

$$
\begin{align}
\text{DFT}_N[(a * y)[n]] &= \text{DFT}_N[(b * x)[n]] \nonumber \\
A[k] Y[k] &= B[k] X[k] \nonumber \\
H[k] = \frac{Y[k]}{X[k]} &= \frac{B[k]}{A[k]} = \frac{ \sum_{l=0}^L b[l]e^{-j \frac{2\pi}{N} nl} }{ \sum_{m=0}^M a[m]e^{-j \frac{2\pi}{N} mk}} \nonumber
\end{align}
$$

que existe siempre que $A[k] \neq 0$. 

La respuesta en frecuencia también suele expresarse como

$$
H[k] = K \frac{ \prod_{l=1}^L (e^{j \frac{2\pi}{N} k}- \beta[l]) }{ \prod_{m=1}^M (e^{j \frac{2\pi}{N} k}- \alpha[m])} 
$$

donde $K$ se llama **ganancia**, las raices del polinomio del numerador $\alpha$ se llaman conjuntamente **ceros** y las raices del polinomio del denominador $\beta$ se llaman conjuntamente **polos**

### Ejemplo: Respuesta al impulso de un sistema IIR

Consideremos el siguiente sistema IIR 

$$
\begin{align}
y[n] &= (1-\gamma) x[n] + \gamma y[n-1] \nonumber \\
y[n] - \gamma y[n-1] &= (1-\gamma) x[n] \nonumber
\end{align}
$$

Los coeficientes del sistema son

$a[0] = 1$, $a[1] = -\gamma$ y $b[0] = (1-\gamma)$

Es decir que es MA de orden 1 y AR de orden 1

¿Cúal es su respuesta al impulso? 

Asumiendo $y[n]=0, n<0$, tenemos que

$$
\begin{matrix}
n & \delta[n] & y[n] \\
-2 & 0 & 0 \\
-1 & 0 & 0 \\
0 & 1 & (1-\gamma) \\
1 & 0 & \gamma(1-\gamma) \\
2 & 0 & \gamma^2(1-\gamma)  \\
3 & 0 & \gamma^3(1-\gamma) \\
4 & 0 & \gamma^4(1-\gamma) \\
\end{matrix}
$$

¿Cómo cambia la respuesta al impulso con distintos valores de $\gamma$? ¿Qué pasa si $\gamma \geq 1$?

In [ ]:
p = [[Figure(plot_width=300, plot_height=230, toolbar_location="below") for k1 in range(3)] for k2 in range(2)]
gamma = [[-0.5, -1, -1.5], [0.5, 1., 1.5]]

for k1 in range(3):
    for k2 in range(2):

        t, y = scipy.signal.dimpulse(([1-gamma[k2][k1], 0], 
                                      [1, -gamma[k2][k1]], 1), x0=0, n=30)
        p[k2][k1].line(t, y[0][:, 0], 
                  line_width=3, legend_label=f"gamma={gamma[k2][k1]}"); 
show(row(column(p[0]), column(p[1])))

Notemos que

- Para $\gamma < 0$ los coeficientes del sistema son alternantes en signo
- Para $|\gamma| < 1$ los coeficientes del sistema tienden a cero
- Para $|\gamma| > 1$ los coeficientes del sistema divergen y tienen a infinito

A diferencia de un sistema FIR el sistema IIR puede tener configuraciones inestables en que los coeficientes crecen o decrecen infinitamente

Consideremos el sistema anterior y asumamos que $|\gamma|<1$, desenrollando tenemos que 

$$
\begin{align}
y[0] &= (1-\gamma) x[0] \nonumber \\
y[1] &= (1-\gamma) (x[1] + \gamma x[0]) \nonumber \\
y[2] &= (1-\gamma) (x[2] + \gamma x[1] + \gamma^2 x[0]) \nonumber \\
y[3] &= (1-\gamma) (x[3] + \gamma x[2] + \gamma^2 x[1] + \gamma^3 x[0]) \nonumber \\
y[4] &= (1-\gamma) (x[4] + \gamma x[3] + \gamma^2 x[2] + \gamma^3 x[1]  + \gamma^4 x[0]) \nonumber \\
y[5] &= \ldots \nonumber 
\end{align}
$$

Con un sistema IIR de 3 coeficientes podemos representar un sistema FIR considerablemente más grande

Por ejemplo si escogemos $\gamma$ tal que $\gamma^{20 }\approx 0$ entonces aproximamos un sistema FIR de orden 20

### Ejemplo: Respuesta en frecuencia de un sistema IIR

Para el sistema del ejemplo anterior su respuesta en frecuencia es

$$
\begin{align}
Y[k] &= (1-\gamma) X[k] + \gamma Y[k] e^{-j \frac{2\pi}{N} k} \nonumber \\
H[k] = \frac{Y[k]}{X[k]} &= \frac{1-\gamma}{1 - \gamma e^{-j \frac{2\pi}{N} k} }  \nonumber 
\end{align}
$$

que en notación de polos y ceros se escribe como

$$
H[k] =  (1-\gamma)\frac{e^{j \frac{2\pi}{N} k} - 0}{e^{j \frac{2\pi}{N} k} - \gamma  }
$$

es decir que tiene un cero en $0$, un polo en $\gamma$ y una ganancia de $(1-\gamma)$

Para entender mejor este sistema estudiemos la magnitud de $|H[k]|$ para $\gamma < 1$

$$
\begin{align}
| H[k]| &= \frac{|1-\gamma|}{|1 - \gamma e^{-j \frac{2\pi}{N} k}|}  \nonumber \\
&=  \frac{1-\gamma}{\sqrt{1 - 2\gamma \cos(\frac{2\pi}{N} k) + \gamma^2}} \nonumber
\end{align}
$$

¿Cómo se ve $|H[k]|$? ¿Qué función cumple este sistema?

In [ ]:
from bokeh.palettes import Dark2_5 as palette

p = Figure(plot_width=500, plot_height=280, toolbar_location="below")

k = np.arange(-24, 25)/50
Hk = lambda gamma, k : (1-gamma)/np.sqrt(1 - 2*gamma*np.cos(2.0*np.pi*k) + gamma**2)

for gamma, color in zip([0.25, 0.5, 0.75], palette):
    p.line(k, Hk(gamma, k), 
           line_width=3, color=color, legend_label=f"gamma={gamma}"); 

p.xaxis.axis_label = 'Frecuencia'
p.yaxis.axis_label = '|H|'
show(p)

Este sistema atenua las frecuencias altas, es decir que actua como un filtro pasa bajos

## Diseño de filtros IIR simples

Los filtros IIR más simples son los de un polo y un cero, es decir filtros de primer orden

$$
H[k] = \frac{b[0] + b[1] e^{-j \frac{2\pi}{N} k}}{1 + a[1] e^{-j \frac{2\pi}{N} k}}  = K\frac{e^{j \frac{2\pi}{N} k} - \beta}{e^{j \frac{2\pi}{N} k} - \alpha  } 
$$

donde podemos reconocer 

- $b[0]=K$
- $\beta = - b[1] \cdot K$
- $\alpha=-a[1]$

Definimos la frecuencia de corte $f_c$ como aquella frecuencia en la que el filtro alcanza una atenuación de 0.7 (-3 dB)

Haciendo la equivalencia con el ejemplo anterior tenemos que $\gamma = e^{-2\pi f_c}$

### Receta para un filtro pasa bajo IIR con frecuencia de corte $f_c$

Asignamos

- $b[0] = 1 - e^{-2\pi f_c}$
- $b[1] = 0$
- $a[1] = -e^{-2\pi f_c}$

Lo que resulta en la siguiente respuesta en frecuencia

$$
H[k] = \frac{1-e^{-2\pi f_c}}{1 - e^{-2\pi f_c} e^{-j \frac{2\pi}{N} k}} = (1-e^{-2\pi f_c}) \frac{(e^{j \frac{2\pi}{N} k}- 0)}{(e^{j \frac{2\pi}{N} k} - e^{-2\pi f_c} )}
$$

Es decir un cero en $0$, un polo en $e^{-2\pi f_c}$ y ganancia $1-e^{-2\pi f_c}$

### Receta para un filtro pasa alto IIR con frecuencia de corte $f_c$

Asignamos

- $b[0] = (1 + e^{-2\pi f_c})/2$
- $b[1] = -(1 + e^{-2\pi f_c})/2$
- $a[1] = -e^{-2\pi f_c}$

Lo que resulta en la siguiente respuesta en frecuencia

$$
H[k] = \frac{1+e^{-2\pi f_c}}{2} \frac{(e^{j \frac{2\pi}{N} k} - 1)}{(e^{j \frac{2\pi}{N} k} - e^{-2\pi f_c})}
$$

Es decir un cero en $1$, un polo en $e^{-2\pi f_c}$ y ganancia $\frac{1+e^{-2\pi f_c}}{2}$



### Aplicar un filtro a una señal con scipy

Para filtrar una señal unidimensional con un filtro IIR podemos utilizar la función


```python
    scipy.signal.lfilter(b, # Coeficientes del numerador
                         a, # Coeficientes del denominador
                         x, # Señal a filtrar
                         ...
                        )
```

Los siguientes ejemplos muestran un señal de tipo pulso rectangular filtrada con sistemas IIR de primer orden pasa bajo y pasa-alto diseñados con las recetas mostradas anteriormente

In [ ]:
def iir_low_pass(signal, fc):
    gamma = np.exp(-2*np.pi*(fc))
    b, a = [(1-gamma), 0], [1, -gamma] 
    return scipy.signal.lfilter(b, a, signal)

def iir_high_pass(signal, fc):
    gamma = np.exp(-2*np.pi*(fc))
    b, a = [(1+gamma)/2, -(1+gamma)/2], [1, -gamma]
    return scipy.signal.lfilter(b, a, signal)

In [ ]:
%%capture

n = np.arange(0, 500)
fig, ax = plt.subplots(2, figsize=(6, 4), tight_layout=True)
l0 = ax[0].plot(n, np.zeros_like(n)); 
ax[0].set_ylim([-0.05, 1.05]); ax[0].set_title('Input')
l1 = [ax[1].plot(n, np.zeros_like(n), label=f'fc={fc}') for fc in [0.01, 0.02, 0.05]]
ax[1].set_ylim([-0.5, 1.05]); ax[1].set_title('Output')
ax[1].legend(loc=4)

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)
    l0[0].set_ydata(x);
    for l, fc in zip(l1, [0.01, 0.02, 0.05]):
        l[0].set_ydata(iir_low_pass(x, fc));

anim = animation.FuncAnimation(fig, update, 
                               frames=200, interval=100, blit=True); 

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
%%capture

n = np.arange(0, 500)
fig, ax = plt.subplots(2, figsize=(6, 4), tight_layout=True)
l0 = ax[0].plot(n, np.zeros_like(n)); 
ax[0].set_ylim([-0.05, 1.05]); ax[0].set_title('Input')
l1 = [ax[1].plot(n, np.zeros_like(n), label=f'fc={fc}') for fc in [0.01, 0.02, 0.05]]
ax[1].set_ylim([-0.5, 1.05]); ax[1].set_title('Output')
ax[1].legend(loc=4)

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)
    l0[0].set_ydata(x);
    for l, fc in zip(l1, [0.01, 0.02, 0.05]):
        l[0].set_ydata(iir_high_pass(x, fc));

anim = animation.FuncAnimation(fig, update, 
                               frames=200, interval=100, blit=True); 

In [ ]:
HTML(anim.to_html5_video())

## Diseño de filtros IIR de segundo orden

Los filtros IIR de segundo orden o **biquad** tienen dos polos y dos ceros.

Su respuesta en frecuencia es

$$
H[k] = \frac{b[0] + b[1] W_N^k + b[2] W_N^{2k}}{1 + a[1] W_N^k + a[2] W_N^{2k}} = K \frac{(W_N^{-k} - \beta_1) (W_N^{-k} - \beta_2)}{(W_N^{-k} - \alpha_1)(W_N^{-k} - \alpha_2)},
$$

donde $W_N = e^{-j \frac{2 \pi}{N}}$ y la relación entreo coeficientes y polos/ceros es: 

$$
b[0] = K, \quad b[1] = -K (\beta_1 + \beta_2), \quad b[2]= K \beta_1\beta_2
$$

$$
a[1] = - (\alpha_1 + \alpha_2), \quad a[2]=\alpha_1 \alpha_2
$$


Con arquitecturas de segundo orden se pueden crear filtros pasabanda y rechaza banda


## Diseño de filtros IIR de orden mayor

Para crear los coeficientes de filtro IIR de orden mayor podemos usar la función

```python
scipy.signal.iirfilter(N, # Orden del filtro
                       Wn,  # Frecuencias de corte (normalizadas en [0,1])
                       fs, # Frecuencia de muestreo
                       btype='bandpass', # Tipo de filtro: 'bandpass', 'lowpass', 'highpass', 'bandstop'
                       ftype='butter', # Familia del filtro: 'butter', 'ellip', 'cheby1', 'cheby2', 'bessel'
                       output='ba', # Retornar coeficientes
                       ...
                       )
```

El filtro Butterworth es óptimo en el sentido de tener la banda de paso lo más plana posible. 

Otros filtros se diseñaron con otras consideraciones. 

Los filtros IIR digitales están basados en los filtros IIR analógicos.

Observe como al aumentar el orden el filtro pasabajo IIR comienza a cortar de forma más abrupta

In [ ]:
from bokeh.palettes import Dark2_5 as palette

p = Figure(plot_width=500, plot_height=280, toolbar_location="below")

for order, color in zip([1, 2, 5, 20], palette):
    b, a = scipy.signal.iirfilter(N=order, Wn=0.2, fs=1,
                                  ftype='butter', btype='lowpass', output='ba')
    freq, response = scipy.signal.freqz(b, a, fs=1)
    p.line(freq, np.absolute(response),
           line_width=3, color=color, legend_label=f"N={order}"); 

p.xaxis.axis_label = 'Frecuencia'
p.yaxis.axis_label = '|H|'
show(p)

## Comparación de la respuesta en frecuencia de filtros FIR e IIR del orden equivalente

Comparemos la respuesta en frecuencia de un filtro IIR y otro FIR ambos con 20 coeficientes

Notemos como el filtro IIR es mucho más abrupto, es decir filtra mejor, que el filtro FIR equivalente

Sin embargo como muestra la columna derecha, el filtro IIR introduce un retardo no lineal a las frecuencias que componen la señal. En general los filtros IIR de orden mayor no preservan la fase

In [ ]:
p = [Figure(plot_width=600, plot_height=250, toolbar_location="below") for k1 in range(2)]

Fs = 1
fc = 0.25
h = scipy.signal.firwin(numtaps=20, cutoff=fc, pass_zero=True, window='hann', fs=Fs)
b, a = scipy.signal.iirfilter(N=10, Wn=fc, fs=Fs, ftype='butter', btype='lowpass')

freq_fir, response_fir = scipy.signal.freqz(h, 1, fs=Fs)
freq_fir, delay_fir = scipy.signal.group_delay(system=(h, 1), fs=Fs)
freq_iir, response_iir = scipy.signal.freqz(b, a, fs=Fs)
freq_iir, delay_iir = scipy.signal.group_delay(system=(b, a), fs=Fs)


p[0].line(freq_fir, np.absolute(response_fir), 
          color=palette[0], line_width=3, legend_label="FIR"); 
p[0].line(freq_iir, np.absolute(response_iir), 
          color=palette[1], line_width=3, legend_label="IIR"); 
p[0].line([fc, fc], [0, 1], color='red', line_width=3, line_dash='dashed'); 

p[1].line(freq_fir, delay_fir, 
          color=palette[0], line_width=3, legend_label="FIR"); 
p[1].line(freq_iir, delay_iir, 
          color=palette[1], line_width=3, legend_label="IIR"); 
p[1].xaxis.axis_label = 'Frecuencia [Hz]'
p[0].yaxis.axis_label = '|H|'
p[1].yaxis.axis_label = 'Desfase [º]'
show(column(p))

## Ejemplo: Efectos de audio


El siguiente ejemplo muestra como implementar el conocido filtro <a href="https://en.wikipedia.org/wiki/Wah-wah_(music)">Wah-wah</a> usando un sistema IIR

Este es un filtro pasabanda modulado con ancho de pasada fijo $f_b$ [Hz] y una frecuencia central variable $f_c$ [Hz], donde La frecuencia central se modula con una onda lenta


Se modela como el siguiente sistema **IIR**

$$
H[k] = \frac{(1+c)W_N^{2k} -(1+c) }{W_N^{2k} + d(1-c)W_N^k -c}
$$

donde 

$$
d=-\cos(2\pi f_c/f_s)
$$ 

y 

$$
c = \frac{\tan(\pi f_b/f_s) -1}{\tan(2\pi f_b /f_s)+1}
$$


In [ ]:
import librosa
data, fs = librosa.load("../../data/nomekop.ogg")
Audio(data, rate=fs)

In [ ]:
data_wah = []
zi = np.zeros(shape=(2,))
# Parámetros fijos del filtro
fb, Nw = 200, 5 
c  = (np.tan(np.pi*fb/fs) - 1.)/(np.tan(2*np.pi*fb/fs) +1)
# Filtramos una ventana de la señal moviendo lentamente fc
for k in range(len(data)//Nw):
    # Cálculo de la frecuencia central
    fc = 500 + 2000*(np.cos(2.0*np.pi*k*30./fs) +1)/2
    d = -np.cos(2*np.pi*fc/fs)
    # Coeficientes del filtro
    b, a = [(1+c), 0, -(1+c)], [1, d*(1-c), -c]
    # Filtramos, usando el filtrado anterior como borde (zi)
    data2, zi = scipy.signal.lfilter(b, a, data[k*Nw:(k+1)*Nw], zi=zi)
    # Guardamos
    data_wah.append(data2)
Audio(np.hstack(data_wah), rate=int(fs))

Más sobre filtros IIR aplicados a efectos de audio: https://www.ee.columbia.edu/~ronw/adst-spring2010/lectures/lecture2.pdf
